In [1]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [8]:
import pandas as pd
from urlparse import urlparse
import tldextract
from collections import Counter
from time import time
import numpy as np

In [220]:
huizhou_stop_words = [u'的', u'了', u'在', u'是', u'安徽', u'和', u'有', u'我', u'年', u'上', u'他', u'我们', u'及',
                      u'为', u'与', u'人', u'也', u'于', u'中', u'就', u'都', u'在线', u'等', u'不', u'网站', u'你', 
                      u'一个', u'到', u'之', u'对', u'说', u'进入', u'后', u'月', u'将', u'时', u'又', u'被', u'会员'
                     u'版权', u'论坛', u'以', u'而', u'着', u'下', u'这', u'但', u'要', u'或', u'首页', u'这', u'更',
                     u'地', u'从', u'卡', u'来', '多', u'日', u'她', u'还', u'联系', u'那', u'一处', u'一直', u'个', 
                     u'向', u'并', u'曾', u'这样', u'里', u'合肥', u'推荐', u'专栏', u'相关', u'当前', u'所有', u'中心'
                     u'频道', u'编辑', u'位置', u'第一', u'社区', u'看', '中安', u'安徽省', u'大', u'许可证', u'许可',
                     u'让', u'没有', u'什么', u'其', '请', '该', '用', '积分', u'凤阳', u'一篇', u'版权', u'县', u'这里',
                      u'就是', u'生于', u'他们', u'它', u'男', u'去', u'会', u'由', u'一', u'做', u'所', u'毕业', u'一种',
                     u'协会会员', u'把', u'能', u'至', u'这个', u'自己', u'黄山市', u'出', u'如', u'如',u'可以', u'很',
                     u'成为', u'好', u'先生', u'中国', u'集团', u'江淮', u'网', u'特色', u'全国', u'新', u'作为', u'师专',
                     u'则', u'重要', u'们', u'给', u'这些', u'过', u'安', u'最', u'现在', u'得', u'国家', u'使', u'这种',
                     u'主要', u'具有', u'人们', u'已', u'以及', u'有着', u'一定', u'一些', u'许多', u'进行', u'由于', u'却', 
                      u'其中', u'只是', u'当时', u'方面', u'即 ', u'徽', u'内容', u'还有', u'之一', u'者 ', u'不仅', u'者',
                      u'特别', u'名', u'提供', u'同时', u'大量', u'因此', u'即']

In [41]:
import unicodedata
def remove_non_chinese(s):
    r = [c for c in s if unicodedata.category(c).startswith('L')]
#print r
    return ''.join(r)
#print remove_non_chinese(s)

In [129]:
def ch_title_letters_removal(x):
    #print 'coming', x
    t = x.strip('  ')
    if(len(t)) == 0:
        return None
    
    if(len(x) > 0):
        #s = x.strip()
        s = remove_non_chinese(x)
        if len(s) > 0 and s not in huizhou_stop_words:
            return s.strip()
        else:
            return None
    return None

In [119]:
df = pd.read_csv('/Users/nhu2000/projects/SimilarSearch/data/clean/huizhou-web-full_v1_4.csv')

In [120]:
df.info()

df = df[df['target'] == 1]

In [51]:
#sampling
msk = np.random.rand(len(df)) < 0.1
df = df[msk]

In [122]:
df.info()

In [123]:
sentences = df['content'].str.decode('utf-8').str.split('  ')

In [124]:
flatten = [item for sublist in sentences for item in sublist]

In [125]:
print len(flatten)

In [126]:
count = Counter(flatten)

In [128]:
top_50 = count.most_common(50)
for t in top_50:
    print t[1]

In [80]:
cnt = 0
ret = []
for item in count.iteritems():
    if item[1] == 1:
        ret.append(item[0])
        cnt += 1

In [81]:
sentences_counted = count.keys()

In [82]:
print len(sentences_counted), type(sentences_counted)

In [83]:
global rr
rr = 0
def map_sentences_index(x):
    global rr
    print 'on going ---', rr
    rr += 1
    s = x.split('  ')
    return [idx for idx, r in enumerate(sentences_counted) if r in s and len(r) != 0]

In [84]:
map_sentences_index(df.iloc[1]['content'])

[1408]

In [85]:
len(sentences_counted[0])

4783

df['s_index'] = df.content.map(lambda x: map_sentences_index(x))

In [86]:
for i, j in enumerate(['foo', 'bar', 'baz']):
    if j == 'bar':
        print i

In [87]:
import jieba
jieba.load_userdict("../data/dict/huizhou_dict.txt")

In [88]:
def jiba_words_separation(x):
    t20 = jieba.cut(x) #搜索引擎模式
    t2 = " ".join(t20)
    return t2

In [90]:
sentences_counted_seg = [jiba_words_separation(item) for item in sentences_counted]

In [91]:
df.info()

In [92]:
words = [item.split() for item in sentences_counted_seg]

In [108]:
flatten = [ch_title_letters_removal(item) for sublist in words for item in sublist if ch_title_letters_removal(item) != None ]

In [109]:
count = Counter(flatten)

In [110]:
top_50 = count.most_common(50)

In [116]:
for t in top_50:
    print t[0], t[1]

In [101]:
sentences_seg = df['content_seg'].str.decode('utf-8')

In [102]:
words_org = [item.split() for item in sentences_seg]

In [221]:
flatten_org = [ch_title_letters_removal(item) for sublist in words_org for item in sublist if ch_title_letters_removal(item) != None ]

In [222]:
count_org = Counter(flatten_org)

In [225]:
top_50_org = count_org.most_common(68)

In [226]:
idx = 0
for t in top_50_org:
    print t[0], t[1]
    idx += 1

In [152]:
vec_dict = dict(top_50_org)

In [227]:
import json
with open('../data/centroids/hz_centroid_68_v1.json', 'w') as fp:
    json.dump(vec_dict, fp)

In [228]:
with open('../data/centroids/hz_centroid_68_v1.json', 'r') as fp:
    v_dict = json.load(fp)

In [229]:
val = v_dict.values()

In [230]:
for w in sorted(v_dict, key=v_dict.get, reverse=True):
    print w, v_dict[w]*1.0/sum(val)

In [157]:
keys = v_dict.keys()

In [158]:
print ' '.join(keys)